## Microsoft에서 제공
### 공식코드 볼수있음
### 최종가중치
### AR-diffusion에 대한 설명
### SST2 dataset -> 감성분류


In [1]:
!pip install transformers
!pip install datasets
!pip install SentencePiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 101.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.2 MB/s eta 0:00:00


In [1]:
import torch
from transformers import XLMProphetNetTokenizer, XLMProphetNetForCausalLM
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset

In [2]:
# Define your custom dataset
class CustomDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length):
      # 변수 초기화
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):# text의 길이
        return len(self.texts)

    def __getitem__(self, idx): # idx 를 사용하여 text목록에서 해당 idx item
        text = str(self.texts[idx])
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask
        }

In [3]:
# Load the XLM-ProphetNet model and tokenizer
model_name = 'microsoft/xprophetnet-large-wiki100-cased'
model = XLMProphetNetForCausalLM.from_pretrained(model_name)
tokenizer = XLMProphetNetTokenizer.from_pretrained(model_name)

In [4]:
# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

XLMProphetNetForCausalLM(
  (prophetnet): XLMProphetNetDecoderWrapper(
    (decoder): XLMProphetNetDecoder(
      (word_embeddings): Embedding(250012, 1024, padding_idx=0)
      (position_embeddings): XLMProphetNetPositionalEmbeddings(512, 1024, padding_idx=0)
      (ngram_embeddings): Embedding(2, 1024)
      (layers): ModuleList(
        (0-11): 12 x XLMProphetNetDecoderLayer(
          (self_attn): XLMProphetNetNgramSelfAttention(
            (key_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (value_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (query_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (relative_pos_embeddings): Linear(in_features=1024, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (cross_attn): XLMProphetNetAttention(


In [5]:
# Define hyperparameters
batch_size = 1 # batchsize 1? -> LLM이기 때문에 2,3인경우 훈련시킬수 없음
max_length = 128
num_epochs = 3
learning_rate = 2e-5

In [6]:
# Load the dataset
## 왜 sst2가아니라 wiki 이걸 썻냐?
dataset = load_dataset('wikitext', 'wikitext-2-raw-v1', split='train[:10]')  # Load the first 100 examples from the WikiText-2 dataset

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Dataset wikitext downloaded and prepared to /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126. Subsequent calls will reuse this data.


In [7]:
dataset

Dataset({
    features: ['text'],
    num_rows: 10
})

In [8]:
# Extract texts from the dataset
texts = dataset['text']

In [16]:
texts

['',
 ' = Valkyria Chronicles III = \n',
 '',
 ' Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n',
 " The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more f

In [9]:
# Create the custom dataset
custom_dataset = CustomDataset(texts, tokenizer, max_length)

# Create the data loader
data_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

In [10]:
# Define the optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

In [11]:
# Training loop
model.train()
for epoch in range(num_epochs):
    total_loss = 0
    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(data_loader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}')

Epoch 1/3, Loss: 11.4653
Epoch 2/3, Loss: 8.6051
Epoch 3/3, Loss: 6.7686


In [12]:
# Inference
model.eval()
example_prompt = 'Once upon a time'
input_ids = tokenizer.encode(example_prompt, return_tensors='pt').to(device)

with torch.no_grad():
    output_ids = model.generate(input_ids, max_length=50)
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    print(f'Example Prompt: {example_prompt}')
    print(f'Generated Text: {generated_text}')

Example Prompt: Once upon a time
Generated Text: Once upon a time
